In [1]:
import pandas as pd
from pathlib import Path
from dataclasses import dataclass, field
import re
from sqlalchemy import create_engine

In [2]:
conn = create_engine('sqlite:///data.db')
cur_dir = Path.cwd()
data_dir = cur_dir / 'data'
if not data_dir.exists:
    data_dir.mkdir()


@dataclass
class Category:
    name: str
    key_words: list[str] = field(default_factory=list)
    transaction: pd.Series = field(default_factory=lambda : pd.Series)
    bank_category: list[str] = field(default_factory=list)


In [8]:
food = Category(
    name='Продукты',
    key_words=['Albert', 'Lidl', 'Kaufland', 'Billa', 'ASIAN MANGO', 'maso',
               'GLOBUS BRNO', 'Potraviny FOLKOVA'])
food_out_side = Category(
    name='Еда на вынос',
    key_words=['VENTANA', 'MOTMOT', 'McDonalds', 'KFC', 'Burger King',
               'Restaurant', 'Restaurace', 'CHILLI TREE', 'KOFIBOX','SUSHIMIX',
               'Damejidlo', 'Pizza', 'NEEXISTUJE', 'KAFE', 'JIDELNI VUZ']
    )
go_pay = Category(
    name='GoPay',
    key_words=['GoPay'])
additional = Category(
    name='Сторонние покупки',
    key_words=['DATART', 'PlayStation', 'CESKA POSTA', 'CINEMA', 'KVETINY',
               'CELIO', 'PRIMARK', 'SB Olympia Brno Brno', 'Kino', 'ROSSMANN',
               'HULK GYM', 'HULK GYM', 'relaxin', 'TIGER', 'ZARA', 'H&M',
               ])
regular_payments = Category(
    name='Регулярные платежи',
    key_words=['hetzner', 'SPOTIFY', 'NETFLIX', 'BARBER', 'BUBELINY', 'Vodafone'],
    bank_category=['Nájem'])
transport = Category(
    name='Транспорт',
    key_words=['DPP', 'Flixbus', 'www.cd.cz', 'MPLA', 'EDALNICE', 'IDSJMK'],
    bank_category=['Doprava',]
)
transfers_n_cache = Category(
    name='Переводы и выдача наличных',
    key_words=['Výber', 'Výběr z bankomatu', 'Revolut', 'MMB2713',
               'VÝBĚR HOTOVOSTI'],
    bank_category=['Bankomat']
)


income = Category(
    name='Переводы на счет',
)
unsorted = Category(
    name='Неотсартированные'
)

categories = [food, food_out_side, go_pay, additional, regular_payments, transport, transfers_n_cache]

In [10]:
df = pd.concat([pd.read_csv(data_dir / f, delimiter=';') for f in data_dir.iterdir()])       
       
columns_2_remove = ['Číslo protiúčtu', 'Banka protiúčtu', 'Název účtu příjemce',
       'Splatnost', 'Variabilní Symbol', 'Specifický Symbol',
       'Konstantní Symbol', 'Zpráva pro příjemce', 'Poznámka pro mě',
       'Název trvalého příkazu', 'Popis platby 2', 'IBAN']
print(df)
df.drop(columns_2_remove, inplace=True, axis=1)

         Číslo účtu                      IBAN Číslo protiúčtu  \
0               NaN                       NaN             NaN   
1    233993963/0600  CZ9306000000000233993963             NaN   
2    233993963/0600  CZ9306000000000233993963             NaN   
3    233993963/0600  CZ9306000000000233993963             NaN   
4    233993963/0600  CZ9306000000000233993963             NaN   
..              ...                       ...             ...   
871  233993963/0600  CZ9306000000000233993963      2457245779   
872  233993963/0600  CZ9306000000000233993963             NaN   
873  233993963/0600  CZ9306000000000233993963             NaN   
874  233993963/0600  CZ9306000000000233993963             NaN   
875  233993963/0600  CZ9306000000000233993963             NaN   

     Banka protiúčtu Název účtu příjemce   Splatnost    Odesláno    Částka  \
0                NaN                 NaN         NaN         NaN   -299,40   
1                NaN                 NaN         NaN  30.01.202

In [ ]:
df['Částka'] = df['Částka'].str.replace(',', '.').astype({'Částka': 'float64'}) 
income.transaction = df[df['Částka'] > 0].copy()
df.drop(income.transaction.index, inplace=True)
df.fillna('No value', inplace=True)
for cat in categories:
    cat.transaction = df[df['Popis platby'].str.contains('|'.join(cat.key_words), flags=re.IGNORECASE, regex=True)].copy()
    df.drop(cat.transaction.index, inplace=True)
    values = df[df['Název kategorie'].isin(cat.bank_category)].copy()
    df.drop(values.index, inplace=True)
    pd.concat([cat.transaction, values])
unsorted.transaction = df

In [ ]:
unsorted.transaction

,Číslo účtu,Odesláno,Částka,Měna,Název kategorie,Typ transakce,Popis platby,Bankovní reference
3,233993963/0600,27.12.2022,-230.0,CZK,Služby,Příkaz k úhradě,OKAMŽITÁ ÚHRADA,0233993963:20221227:00009:221226XSB4157917905
13,233993963/0600,27.12.2022,-280.0,CZK,Bydlení,Použití platební karty,MOBILNI STANEK 4 PONIKLA CZ,0233993963:20221227:00001:221223V0176275
48,233993963/0600,13.12.2022,-390.0,CZK,Jídlo a pití,Použití platební karty,SUMUP *DOMOVINA BRNO CZ,0233993963:20221213:00006:221212V1113569


In [ ]:
unsorted.transaction.columns

Index(['Číslo účtu', 'Odesláno', 'Částka', 'Měna', 'Název kategorie',
       'Typ transakce', 'Popis platby', 'Bankovní reference'],
      dtype='object')

In [ ]:
additional.transaction.to_sql('transaction', conn, if_exists='append', index=False)

15

In [13]:
from datetime import datetime
from pandas import date_range
from datetime import date, timedelta
    
month = datetime.now().month + 1
next_month = month + 1

start_date = date(2023, month, 1) 
end_date = date(2023, month + 1, 1)    # perhaps date.now()
next_month = start_date.replace(day=28) + timedelta(days=4)
res = next_month - timedelta(days=next_month.day)

delta = res - start_date   # returns timedelta

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    print(day)


2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-11
2023-03-12
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
2023-03-19
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-25
2023-03-26
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31


In [9]:
print(dr)

[Timestamp('2023-01-02 00:00:00', freq='D'), Timestamp('2023-01-03 00:00:00', freq='D')]
